In [1]:
import torch.nn as nn
import torch
import numpy as np

## Solve for x from y and w
$$x = y * W$$

In [ ]:
def solve_for_x(y, weights, size_K = 2):
  H, W = y.shape[0], y.shape[1]
  k_H, k_W  =  size_K, size_K
  x = y.clone()
  for h in range(H):
    for w in range(W):
      for k_h in range(k_H):
        if h-k_h<0:
          break
        for k_w in range(k_W):
          if w-k_w<0:
            break
          x[h,w] = x[h,w] - x[h-k_h, w-k_w] * weights[k_H-k_h-1, k_W-k_w-1]
  return x
          

In [ ]:
def solve_parallel(y, weights, size_K):
    H, W = y.shape
    x = y.clone()
    k_H, k_W = size_K[0], size_K[1] # height od kernel 
    if not H % 2 and W % 2: 
        n_steps = 2 * W # diagonal
    else:
        n_steps = 2 * W - 1 # diagonal
    n_parallel_op = min(H, W) # of parallel operations 
    for i in range(n_steps):
      for j in range(max(H, W)):
          if j > i:
              break
          pixel = (j, i - j)
          h = pixel[0]
          w = pixel[1]
          if h >= H or w >= W:
              continue 
          M_row = h * W + w
          for k_h in range(k_H):
              if h - k_h < 0:
                  break
              for k_w in range(k_W):
                  if w - k_w < 0:
                      break
                  x[h, w] -= x[h - k_h, w - k_w] * \
                                            weights[ k_H - k_h - 1, k_W - k_w - 1]
    return x

In [2]:
n = 3
k = 3
m = nn.Conv2d(1, 1, k, bias=False, stride=1, padding="same")
target = torch.randn(1, 1, n, n)
weight = np.random.randn(k,k)
m.weight = nn.Parameter(torch.FloatTensor(weight).reshape(shape=(1,1,k,k))).requires_grad_()
input = np.random.randn(n,n)
# input = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.float32)
input_t = torch.from_numpy(input).float()
input_t = input_t.reshape(shape=(1,1,n,n)).requires_grad_()

# m.bias = nn.Parameter(torch.zeros(1))
output = m(input_t)
# out =  target - output

In [3]:
m.bias

In [4]:
def convolve(img: np.array, kernel: np.array, output_size: int) -> np.array:
    # Assuming a rectangular image
    # tgt_size = calculate_target_size(
    #     img_size=img.shape[0],
    #     kernel_size=kernel.shape[0]
    # )
    # To simplify things
    tgt_size = output_size
    k = kernel.shape[0]
    
    # 2D array of zeros
    convolved_img = np.zeros(shape=(tgt_size, tgt_size))
    
    # Iterate over the rows
    for i in range(tgt_size):
        # Iterate over the columns
        for j in range(tgt_size):
            # img[i, j] = individual pixel value
            # Get the current matrix
            mat = img[i:i+k, j:j+k]
            
            # Apply the convolution - element-wise multiplication and summation of the result
            # Store the result to i-th row and j-th column of our convolved_img array
            convolved_img[i, j] = np.sum(np.multiply(mat, kernel))
            
    return convolved_img

In [5]:
img_pad = np.pad(input, ((1, 1), (1, 1)), 'constant')
# print(img_pad)
output1 = convolve(img_pad, weight, output_size=n)

In [6]:
loss_t = (output*output).sum()

In [7]:
print(output.shape, output1.shape)
print(output)
print(output1)
print(np.abs((output.detach().numpy()[0,0] - output1)).sum())

torch.Size([1, 1, 3, 3]) (3, 3)
tensor([[[[ 1.5014, -0.6445,  0.2733],
          [ 1.0038, -1.1332,  0.4172],
          [ 0.2419, -0.3117,  1.1860]]]], grad_fn=<ConvolutionBackward0>)
[[ 1.50137474 -0.64445432  0.2733294 ]
 [ 1.00376013 -1.13321239  0.41722132]
 [ 0.24192223 -0.31171648  1.1860388 ]]
3.7124478075023504e-07


In [8]:
output1_mean = output1.sum()
output_mean = output.sum()
print(output_mean, output_mean)

tensor(2.5343, grad_fn=<SumBackward0>) tensor(2.5343, grad_fn=<SumBackward0>)


In [9]:
# loss = np.sum(output-image)

error1  = target - output1
error = target - output
L = (error1*error1)/2
dL_dy = 2*output1
print(error1.sum())
print(error.sum())


tensor(-6.5657, dtype=torch.float64)
tensor(-6.5657, grad_fn=<SumBackward0>)


In [10]:
loss_t.backward() # backprop for nn.con2d

In [ ]:
# dL_dy_pad = np.pad(dL_dy, 1, 'constant', 0)
# dL_dy_pad = np.pad(error1.detach().numpy()[0,0], ((1, 1), (1, 1)), 'constant', constant_values=((0,0), (0,0)))
# print(dL_dy_pad)

In [ ]:
# print(weight)

In [11]:
weight_rot = weight[::-1, ::-1]
# weight_rot

In [12]:

dL_dy_pad = np.pad(dL_dy, ((1,2),(1,1)),'constant')
# weight_rot_pad= np.pad(weight_rot, ((2, 2), (2, 2)), 'constant')
# print(weight)
# dL_dy = 
print('/n')
print(input_t.grad)
# dL_dy_2d = dL_dy
# print(dL_dy)
dL_dx = convolve(dL_dy_pad, weight_rot, output_size=n)
print(dL_dx)

/n
tensor([[[[-1.6696,  1.8035, -0.1010],
          [-0.8668,  3.1325, -1.1151],
          [-0.1356,  0.5607, -1.6699]]]])
[[-1.66961437  1.80348426 -0.10101961]
 [-0.86679274  3.13253995 -1.11512261]
 [-0.13559183  0.5607173  -1.66986351]]


In [ ]:
# print(output.grad)

In [13]:
# error_pad = error1.detach().numpy()[0,0]
# error_pad = np.pad(error_pad, ((1, 1), (1, 1)), 'constant', constant_values=((0,0), (0,0)))
# img_pad2
# input_pad = np.pad(input, ((2, 2), (2, 2)), 'constant')
dL_dw = convolve(dL_dy_pad, input, output_size = 3)
# print(error1.detach().numpy()[0,0].shape)
print('dL_dw:', dL_dw)
print('m.weight.grad',m.weight.grad)

dL_dw: [[ 10.69583565  -8.6539254    4.8907915 ]
 [  9.97971688 -13.68672992   7.03434578]
 [  4.31319354  -7.04011235   8.89302201]]
m.weight.grad tensor([[[[  8.8930,  -7.0401,   4.3132],
          [  7.0343, -13.6867,   9.9797],
          [  4.8908,  -8.6539,  10.6958]]]])


In [14]:
weight_grad = m.weight.grad.detach().numpy()[0,0]
weight_grad = weight_grad[::-1,::-1]
# print(weight_grad)
print(np.abs((weight_grad - dL_dw)).sum())


5.789721651439095e-06


tensor([[[[-4.9229, -2.6215, -3.0067],
          [-2.7976, -0.2435, -2.1570],
          [ 1.6515,  2.7533,  1.0473]]]])

In [ ]:
W = np.ones(shape=(k,k)).reshape((1,1,k,k))

In [ ]:
print(W)

[[[[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]]]


In [ ]:
nn.Paramteters(torch.tensor(W))

In [ ]:
# print(output.grad_fn)
# loss = nn.CrossEntropyLoss()
# target = np.random.randn(n,n)
# target = torch.from_numpy(target)
# print(target.shape)
# output_ = loss(output, target)
# print(grad_in[1])

In [ ]:
# def backward_hook(m, input_gradients, output_gradients):
#     print('input_gradients {}'.format(input_gradients))
#     print('output_gradients {}'.format(output_gradients))
#     input_gradients = (torch.ones_like(input_gradients[0]), )
#     return input_gradients

# conv = nn.Conv2d(1, 1, 3)
# conv.register_full_backward_hook(backward_hook)

# x = torch.randn(1, 1, 3, 3).requires_grad_()

# out = m(input_t)

# out =  target - output
# out.mean().backward()
 # ones # print the gradient using .grad
# # print("A.grad:", input_t.grad)

In [ ]:
# loss = nn.CrossEntropyLoss()
# output_ = loss(target, output)

# loss_ = 

In [ ]:
# input = torch.randn(20, 2, 50, 50)

In [ ]:
import cv2 
import numpy as np

In [ ]:
class Conv_Module():
    def __init__(self, img, ker, b, pad=1, stride=1):            #initialization
        
        self.img_b, self.img_d, self.img_h, self.img_w=img.shape
        self.ker_b, self.ker_d, self.ker_h, self.ker_w=ker.shape
        self.b, self.pad, self.stride, self.img, self.ker=b, pad, stride, img, ker
        
    def __call__(self):                                      
        self.out=self.forward(self.img, self.ker)            #forward method is called with images, kernel as input
        return self.out
    
    def forward(self): raise Exception('Not Implemented')    #default raises not implemented error.
    def backward(self): return self.bwd(self.out)           #output from the forward method gets passed here as the input.


In [ ]:

# input = np.arange(3,3)
# image = np.array([[1, 2, 3], [ 4, 5, 6], [7, 8, 9]])
# input = np.random.rand(5,5)

# input = np.array([[0, 0, 0, 0,0], [0, 1, 2, 3, 0], [0, 4, 5, 6, 0], [0, 7, 8, 9, 0], [0, 0, 0, 0, 0]])
# weight = np.array([[-1, -1, -1], [-1, 0, -1], [-1, -1, -1]]) #np.array([[1, 2], [3, 4]])
weight = m.weight.detach().numpy()[0,0]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
img=np.random.rand(1, 1, 10,10)
ker=np.random.rand(1, 1, 3,3)

In [ ]:
print(input.shape)
print(weight.shape)

(5, 5)
(3, 3)


In [ ]:
m.weight.shape

NameError: ignored

In [ ]:
# loss = np.sum(output-image)
dL_dy = 2*(input-output1)
print(dL_dy)

[[-1.5564955  -2.46030054 -2.21440802 -1.17244977 -0.21756689]
 [-5.23405529 -4.39268759 -1.98241121 -3.59447544 -0.47502387]
 [-3.75203804 -2.77445371 -1.72026749 -2.55610292 -1.48946767]
 [-3.08942276 -2.37246138 -2.04105977 -3.28936444 -0.30321979]
 [-2.33481116 -1.81979396 -0.3374369  -2.88452972 -0.42143236]]


In [ ]:
# dL/dy = 
# dL/dx =
# dL/dw = 

In [ ]:
m_test = torch.nn.Sigmoid()
loss_test = nn.BCELoss()
input_test = torch.randn(3, requires_grad=True)
target_test = torch.empty(3).random_(2)
output_test = loss_test(m_test(input_test), target_test)
output_test.backward()

In [ ]:
output_test

tensor(1.2533, grad_fn=<BinaryCrossEntropyBackward0>)

In [ ]:
input_test

tensor([-1.6157,  0.2495,  0.7506], requires_grad=True)

In [ ]:
target_test

tensor([1., 0., 0.])

In [ ]:
m_test(input_test)

tensor([0.1658, 0.5620, 0.6793], grad_fn=<SigmoidBackward0>)